In [2]:
import pgmpy 
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
import pandas as pd
import numpy as np

In [3]:
url = 'https://raw.githubusercontent.com/wshenyc/MLForCities/main/Data/housing_data.csv'
df = pd.read_csv(url)
#df.set_index("bbl", inplace = True)
# df.set_index("cd", inplace = True)
# df = df.replace('?',np.nan)
# df.dropna(inplace=True, how='any')

In [4]:
df['outcome'] = df['outcome'].replace([False, True], [0,1])
df['outcome'].mean()

0.125858962607074

In [5]:
df.isna().sum()

bbl                       0
cd                       56
county                    0
tract10                  56
res_units                 0
other_units               0
year_built               90
year_reno                65
buildings                 0
floors                  306
building_class            0
basement_code             0
owner_type           153734
lot_area                  0
res_sqft                713
comp_bbl_2017             0
comp_bbl_2018             0
comp_bbl_2019             0
comp_bbl_2020             0
comp_bbl_2021             0
viol_bbl_oth_2017         0
viol_bbl_oth_2018         0
viol_bbl_oth_2019         0
viol_bbl_oth_2020         0
viol_bbl_oth_2021         0
viol_bbl_ser_2017         0
viol_bbl_ser_2018         0
viol_bbl_ser_2019         0
viol_bbl_ser_2020         0
viol_bbl_ser_2021         0
outcome                   0
dtype: int64

In [8]:
#gonna need to clean this later
#dropping columns with NA values 
df = df.dropna(axis='columns')
df_prepped = pd.get_dummies(df)
df_prepped.head()

,bbl,county,res_units,other_units,buildings,basement_code,lot_area,comp_bbl_2017,comp_bbl_2018,comp_bbl_2019,...,building_class_W2,building_class_W3,building_class_W4,building_class_W6,building_class_W7,building_class_W8,building_class_W9,building_class_Y4,building_class_Z4,building_class_Z9
0,3047940001,47,4,3,2,5,4000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,3047940017,47,3,0,1,2,2000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3047940078,47,3,0,1,2,1400,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2032790058,5,6,0,1,5,2703,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,1011980126,61,7,0,1,1,1916,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
s = df_prepped.pop('outcome')
df_prepped_v2 = pd.concat([df_prepped, s], 1)
df_prepped_v2.head()

/var/folders/01/r5t37nc50sz2v6yhb4k91mzw0000gn/T/ipykernel_55837/1443122164.py:2: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_prepped_v2 = pd.concat([df_prepped, s], 1)


,bbl,county,res_units,other_units,buildings,basement_code,lot_area,comp_bbl_2017,comp_bbl_2018,comp_bbl_2019,...,building_class_W3,building_class_W4,building_class_W6,building_class_W7,building_class_W8,building_class_W9,building_class_Y4,building_class_Z4,building_class_Z9,outcome
0,3047940001,47,4,3,2,5,4000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,3047940017,47,3,0,1,2,2000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3047940078,47,3,0,1,2,1400,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,2032790058,5,6,0,1,5,2703,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0
4,1011980126,61,7,0,1,1,1916,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# df_prepped_v2 = df_prepped_v2.drop(columns='viol_bbl_ser_2021')
list(df_prepped_v2.columns)
# df_prepped_v2.head()

['bbl',
 'county',
 'res_units',
 'other_units',
 'buildings',
 'basement_code',
 'lot_area',
 'comp_bbl_2017',
 'comp_bbl_2018',
 'comp_bbl_2019',
 'comp_bbl_2020',
 'comp_bbl_2021',
 'viol_bbl_oth_2017',
 'viol_bbl_oth_2018',
 'viol_bbl_oth_2019',
 'viol_bbl_oth_2020',
 'viol_bbl_oth_2021',
 'viol_bbl_ser_2017',
 'viol_bbl_ser_2018',
 'viol_bbl_ser_2019',
 'viol_bbl_ser_2020',
 'building_class_A8',
 'building_class_B1',
 'building_class_C0',
 'building_class_C1',
 'building_class_C2',
 'building_class_C3',
 'building_class_C4',
 'building_class_C5',
 'building_class_C7',
 'building_class_C9',
 'building_class_CM',
 'building_class_D1',
 'building_class_D2',
 'building_class_D3',
 'building_class_D5',
 'building_class_D6',
 'building_class_D7',
 'building_class_D8',
 'building_class_D9',
 'building_class_E1',
 'building_class_E9',
 'building_class_F4',
 'building_class_F5',
 'building_class_F9',
 'building_class_G0',
 'building_class_G1',
 'building_class_G2',
 'building_class_G9',
 '

In [12]:
df_prepped_v2.to_csv("building_data_final.csv")

In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

X = df_prepped_v2.loc[:, "bbl":"building_class_Z9"]
y = df_prepped_v2.loc[:, 'outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=999)

dt = DecisionTreeClassifier(max_depth=10, random_state = 99)
dt.fit(X_train, y_train)
y_predict = dt.predict(X_test)

# accuracy is very high
accuracy= (y_predict == y_test).value_counts(normalize=True)[True]
accuracy

0.9549266689143627

In [55]:
pred_prob= dt.predict_proba(X_test)[:,1]

In [56]:
pred_prob_allbld= pd.concat([df_prepped_v2.loc[:, ["bbl"]], 
                            pd.DataFrame(dt.predict_proba(df_prepped_v2.loc[:, "bbl":"building_class_Z9"])[:, 1],
                                        columns = ["prob_vio_dt"])], axis = 1)
pred_prob_allbld

,bbl,prob_vio_dt
0,3047940001,0.060268
1,3047940017,0.001839
2,3047940078,0.001839
3,2032790058,0.157187
4,1011980126,0.011708
...,...,...
158180,1011980055,0.001839
158181,2039340058,0.001839
158182,3087140010,0.947761
158183,2027860012,0.800000


In [57]:
def trainGaussianNaiveBayes(X,y):
    prior = 1.*y.sum()/y.count()
    nbc = {'prior':prior}
    X_1 = X[y==1]
    X_0 = X[y==0]
    for j in X.columns:
        nbc[j+'_mu1'] = X_1[j].mean()
        nbc[j+'_sigma1'] = X_1[j].std()
        nbc[j+'_mu0'] = X_0[j].mean()
        nbc[j+'_sigma0'] = X_0[j].std()
    return nbc

def testGaussianNaiveBayes(X,nbc):
    y_pred = pd.Series(index=X.index,dtype='float64')
    for i in X.index:
        # compute odds of y=1
        y_pred[i] = nbc['prior']/(1-nbc['prior']) # prior odds
        for j in X.columns:
            thevalue = X.loc[i,j]
            pdf1 = stats.norm.pdf(thevalue,loc=nbc[j+'_mu1'],scale=nbc[j+'_sigma1'])
            pdf0 = stats.norm.pdf(thevalue,loc=nbc[j+'_mu0'],scale=nbc[j+'_sigma0'])
            y_pred[i] = y_pred[i]*pdf1/pdf0 if pdf0 > 0 else 1E10
        # convert odds to probability of y=1
        y_pred[i] = y_pred[i]/(1.0+y_pred[i])
    return y_pred

In [63]:
import scipy.stats
from scipy import stats


In [68]:
list(df_prepped_v2.columns)

['bbl',
 'county',
 'res_units',
 'other_units',
 'buildings',
 'basement_code',
 'lot_area',
 'comp_bbl_2017',
 'comp_bbl_2018',
 'comp_bbl_2019',
 'comp_bbl_2020',
 'comp_bbl_2021',
 'viol_bbl_oth_2017',
 'viol_bbl_oth_2018',
 'viol_bbl_oth_2019',
 'viol_bbl_oth_2020',
 'viol_bbl_oth_2021',
 'viol_bbl_ser_2017',
 'viol_bbl_ser_2018',
 'viol_bbl_ser_2019',
 'viol_bbl_ser_2020',
 'building_class_A8',
 'building_class_B1',
 'building_class_C0',
 'building_class_C1',
 'building_class_C2',
 'building_class_C3',
 'building_class_C4',
 'building_class_C5',
 'building_class_C7',
 'building_class_C9',
 'building_class_CM',
 'building_class_D1',
 'building_class_D2',
 'building_class_D3',
 'building_class_D5',
 'building_class_D6',
 'building_class_D7',
 'building_class_D8',
 'building_class_D9',
 'building_class_E1',
 'building_class_E9',
 'building_class_F4',
 'building_class_F5',
 'building_class_F9',
 'building_class_G0',
 'building_class_G1',
 'building_class_G2',
 'building_class_G9',
 '

In [77]:
X = df_prepped_v2.loc[:, "county":"building_class_Z9"].fillna(0) #less columns
y = df_prepped_v2.loc[:, "outcome"].fillna(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

In [75]:
naive_bayes_classifier = trainGaussianNaiveBayes(X_train,y_train)
#y_pred_test = testGaussianNaiveBayes(X_test,naive_bayes_classifier)
#print('Out of sample prediction accuracy:',1.0*sum((y_pred_test>0.5)==y_test)/len(y_test))

In [80]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
trained_model = gnb.fit(X_train,y_train)
# y_pred_train = trained_model.predict_proba(X_train)[:,1]
# y_pred_test = trained_model.predict_proba(X_test)[:,1]

In [89]:
y_pred_all2 = trained_model.predict_proba(X)[:,1]

In [102]:
pred_prob_allbld_gaus= pd.concat([df_prepped_v2.loc[:, ["bbl"]], 
                            pd.DataFrame(y_pred_all2,
                                        columns = ["prob_viol_bbl"])], axis = 1)
pred_prob_allbld_gaus

,bbl,prob_viol_bbl
0,3047940001,1.372550e-11
1,3047940017,8.803477e-12
2,3047940078,1.296948e-11
3,2032790058,3.000307e-11
4,1011980126,8.220811e-11
...,...,...
158180,1011980055,9.234683e-12
158181,2039340058,1.152486e-11
158182,3087140010,1.000000e+00
158183,2027860012,3.203259e-07


In [81]:
# measure accuracy for the binary prediction task
print('In sample prediction accuracy:',1.0*sum((y_pred_train>0.5)==y_train)/len(y_train))
print('Out of sample prediction accuracy:',1.0*sum((y_pred_test>0.5)==y_test)/len(y_test))


In sample prediction accuracy: 0.9125322409346078
Out of sample prediction accuracy: 0.9111689887981389


In [88]:
#y_pred_all = testGaussianNaiveBayes(X,naive_bayes_classifier)
X.head()

,county,res_units,other_units,buildings,basement_code,lot_area,comp_bbl_2017,comp_bbl_2018,comp_bbl_2019,comp_bbl_2020,...,building_class_W2,building_class_W3,building_class_W4,building_class_W6,building_class_W7,building_class_W8,building_class_W9,building_class_Y4,building_class_Z4,building_class_Z9
0,47,4,3,2,5,4000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,47,3,0,1,2,2000,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,47,3,0,1,2,1400,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,5,6,0,1,5,2703,0.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,61,7,0,1,1,1916,0.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
pred_prob_allbld_gaus= pd.concat([df_prepped_v2.loc[:, ["bbl"]], 
                            pd.DataFrame(y_pred_all2,
                                        columns = ["prob_viol_bbl"])], axis = 1)
pred_prob_allbld_gaus

,bbl,prob_viol_bbl
0,3047940001,1.372550e-11
1,3047940017,8.803477e-12
2,3047940078,1.296948e-11
3,2032790058,3.000307e-11
4,1011980126,8.220811e-11
...,...,...
158180,1011980055,9.234683e-12
158181,2039340058,1.152486e-11
158182,3087140010,1.000000e+00
158183,2027860012,3.203259e-07


In [108]:
print("Decision Tree->", "BBL:", pred_prob_allbld['bbl'][158182], "Prob:", pred_prob_allbld['prob_vio_dt'][0] )
print("Naive Bayes->", "BBL:", pred_prob_allbld_gaus['bbl'][158182], "Prob:", pred_prob_allbld_gaus['prob_viol_bbl'][0])


Decision Tree-> BBL: 3087140010 Prob: 0.060267857142857144
Naive Bayes-> BBL: 3087140010 Prob: 1.3725496674756723e-11


In [ ]:
#snipping code chunks for bayes net
# get the conditional distributions
from pgmpy.inference import VariableElimination
infer = VariableElimination(model)

d= {}
for i in range(4):
    for j in range(4):
        for k in range(1,5):
            # print conditional distributions
            if train.loc[(train["mediangrossrent"]==j) & (train["landuse_cat"]==k)&(train["unitsres"]==i)].shape[0] > 0:
                print("\nunitsres=",i)
                print("mediangrossrent=", j)
                print("landuse_cat=", k)
                cpd_comp= infer.query(['vacate_ind'], evidence={'unitsres': i, 'mediangrossrent': j, 'landuse_cat': k})
                d["unitsres="+str(i)+"; mediangrossrent="+str(j)+"; landuse_cat="+str(k)]= cpd_comp.values[1]
                print(cpd_comp) 

In [ ]:
#####only look at code above this line lol 

In [4]:
assert df["bbl"].isna().sum() == 0
#assert df_bbl["holc_D"].isna().sum() == 0
print(df.columns)
df.describe()

Index(['bbl', 'cd', 'county', 'tract10', 'res_units', 'other_units',
       'year_built', 'year_reno', 'buildings', 'floors', 'building_class',
       'basement_code', 'owner_type', 'lot_area', 'res_sqft', 'comp_bbl_2017',
       'comp_bbl_2018', 'comp_bbl_2019', 'comp_bbl_2020', 'comp_bbl_2021',
       'viol_bbl_oth_2017', 'viol_bbl_oth_2018', 'viol_bbl_oth_2019',
       'viol_bbl_oth_2020', 'viol_bbl_oth_2021', 'viol_bbl_ser_2017',
       'viol_bbl_ser_2018', 'viol_bbl_ser_2019', 'viol_bbl_ser_2020',
       'viol_bbl_ser_2021', 'outcome'],
      dtype='object')


,bbl,cd,county,tract10,res_units,other_units,year_built,year_reno,buildings,floors,...,viol_bbl_oth_2017,viol_bbl_oth_2018,viol_bbl_oth_2019,viol_bbl_oth_2020,viol_bbl_oth_2021,viol_bbl_ser_2017,viol_bbl_ser_2018,viol_bbl_ser_2019,viol_bbl_ser_2020,viol_bbl_ser_2021
count,1.581850e+05,158129.000000,158185.000000,1.581290e+05,158185.000000,158185.000000,158095.000000,158120.000000,158185.000000,157879.000000,...,158185.000000,158185.000000,158185.000000,158185.000000,158185.000000,158185.000000,158185.000000,158185.000000,158185.000000,158185.000000
mean,2.892668e+09,292.963637,52.117236,5.215384e+07,12.557297,0.658621,1935.024112,1950.007975,1.209476,3.550281,...,0.958992,1.111887,0.998713,0.478929,1.481100,0.187354,0.204266,0.236416,0.194892,0.252430
std,9.936538e+08,98.099143,23.485196,2.349119e+07,66.758475,44.070359,35.759542,40.588396,5.159524,2.827315,...,3.171107,3.475358,3.355485,2.516292,3.255035,1.066877,1.142046,1.274121,1.099580,1.367373
min,1.000070e+09,101.000000,5.000000,5.000200e+06,3.000000,-25.000000,1789.000000,1789.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.046770e+09,211.000000,47.000000,4.702000e+07,3.000000,0.000000,1910.000000,1920.000000,1.000000,2.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.032650e+09,307.000000,47.000000,4.705890e+07,4.000000,0.000000,1928.000000,1931.000000,1.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.005480e+09,401.000000,81.000000,8.100520e+07,8.000000,0.000000,1955.000000,1989.000000,1.000000,4.000000,...,1.000000,1.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.080470e+09,503.000000,85.000000,8.503230e+07,19201.000000,17356.000000,2022.000000,2022.000000,1856.000000,98.000000,...,157.000000,175.577452,164.000000,131.571429,139.675676,60.800000,70.666667,85.000000,64.000000,133.891892


In [31]:
from sklearn.cluster import KMeans

df2 = df
df2.iloc[:,0:] = df2.iloc[:,0:].apply(pd.to_numeric,errors='coerce',downcast='integer')
#df2 = df2.dropna(how='any')
df2 = df2.dropna(axis="columns")


KM = KMeans(n_clusters=3,random_state=522)
scor=KM.fit_predict(df2)

array([1, 1, 0, ..., 0, 2, 2], dtype=int32)

In [38]:
res_km=pd.DataFrame(KM.transform(df2))
res_km.index = df2.index
res_km.columns=list(range(3))
res_km["nearest_dist"] = res_km.min(axis=1)
res_km['cluster'] = res_km.idxmin(axis=1)
test = res_km.sort_values(by= ["nearest_dist"], ascending=False).head(5)
test

,0,1,2,nearest_dist,cluster
cd,,,,,
503.0,4.054593e+09,1.871315e+09,3.041048e+09,1.871315e+09,1
503.0,4.053033e+09,1.869755e+09,3.039489e+09,1.869755e+09,1
503.0,4.050793e+09,1.867515e+09,3.037249e+09,1.867515e+09,1
502.0,4.027503e+09,1.844229e+09,3.013967e+09,1.844229e+09,1
503.0,4.027173e+09,1.843899e+09,3.013638e+09,1.843899e+09,1


In [ ]:
from collections import defaultdict
def temp_predict(model,data):
    missing_variables = list(set(model.nodes()) - set(data.columns))
    pred_values = defaultdict(list)
    model_inference = VariableElimination(model)
    for index, data_point in data.iterrows():
        states_dict = model_inference.map_query(variables=missing_variables, evidence=data_point.astype(int).to_dict())
        for k, v in states_dict.items():
            pred_values[k].append(v)
    return pd.DataFrame(pred_values, index=data.index)

train_x = train.loc[:,"tip":"pass"]
#predicted_train = best_model.predict(train_x) should work, but is buggy in the current version
predicted_train = temp_predict(best_model, train_x)
print "In sample:",(train.loc[:,'manhattan']==predicted_train['manhattan']).mean()

test_x = test.loc[:,"tip":"pass"]
#predicted_test = best_model.predict(test_x) should work, but is buggy in the current version
predicted_test = temp_predict(best_model, test_x)
print "Out of sample:",(test.loc[:,'manhattan']==predicted_test['manhattan']).mean()